In [2]:
## Importing Libraries from Python ##  -  Import all libraries to be used here.
import pandas as pd
import numpy as np
import spacy

In [4]:
## Read the dataset using delimitter '\t' ##
## Also print the dataset ##
DataSet = pd.read_csv('amazon_alexa_TEST.tsv',sep = '\t')
print(DataSet)
DataSet.shape

      rating       date         variation  \
0          5  31-Jul-18  Charcoal Fabric    
1          5  31-Jul-18  Charcoal Fabric    
2          4  31-Jul-18    Walnut Finish    
3          5  31-Jul-18  Charcoal Fabric    
4          5  31-Jul-18  Charcoal Fabric    
...      ...        ...               ...   
3145       5  30-Jul-18        Black  Dot   
3146       5  30-Jul-18        Black  Dot   
3147       5  30-Jul-18        Black  Dot   
3148       5  30-Jul-18        White  Dot   
3149       4  29-Jul-18        Black  Dot   

                                       verified_reviews  feedback  
0                                         Love my Echo!         1  
1                                             Loved it!         1  
2     Sometimes while playing a game, you can answer...         1  
3     I have had a lot of fun with this thing. My 4 ...         1  
4                                                 Music         1  
...                                                

(3150, 5)

In [7]:
## Describing the dataset using describe() ##
## This provides us information about the dataset ##
DataSet.describe()

,rating,feedback
count,3150.000000,3150.000000
mean,4.463175,0.918413
std,1.068506,0.273778
min,1.000000,0.000000
25%,4.000000,1.000000
50%,5.000000,1.000000
75%,5.000000,1.000000
max,5.000000,1.000000


In [8]:
## Print the first 5 rows of the dataset for viewing the details of each row and column ##
DataSet.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [9]:
## Word Tokenization

nlp = spacy.load("en_core_web_sm")
for row in range(len(DataSet)):
    Reviews = DataSet.loc[row, 'verified_reviews']
    doc = nlp(Reviews)
    for token in doc:
        print(token.text)

Love
my
Echo
!
Loved
it
!
Sometimes
while
playing
a
game
,
you
can
answer
a
question
correctly
but
Alexa
says
you
got
it
wrong
and
answers
the
same
as
you
.
 
I
like
being
able
to
turn
lights
on
and
off
while
away
from
home
.
I
have
had
a
lot
of
fun
with
this
thing
.
My
4
yr
old
learns
about
dinosaurs
,
i
control
the
lights
and
play
games
like
categories
.
Has
nice
sound
when
playing
music
as
well
.
Music
I
received
the
echo
as
a
gift
.
I
needed
another
Bluetooth
or
something
to
play
music
easily
accessible
,
and
found
this
smart
speaker
.
Ca
n’t
wait
to
see
what
else
it
can
do
.
Without
having
a
cellphone
,
I
can
not
use
many
of
her
features
.
I
have
an
iPad
but
do
not
see
that
of
any
use
.
 
It
IS
a
great
alarm
.
 
If
u
r
almost
deaf
,
you
can
hear
her
alarm
in
the
bedroom
from
out
in
the
living
room
,
so
that
is
reason
enough
to
keep
her
.
It
is
fun
to
ask
random
questions
to
hear
her
response
.
 
She
does
not
seem
to
be
very
smartbon
politics
yet
.
I
think
this
is
the
5th
one
I
've

The
sound
is
amazing
and
many
collections
of
all
music
Setting
up
She
does
n’t
always
listen
It
’s
awesome
I
bought
this
because
it
can
control
fios
television
.
It
works
well
.
I
’m
hoping
over
time
it
will
do
more
to
control
fios
.
Then
I
discovered
all
the
other
things
Alexa
can
do
.
Play
music
you
ask
for
.
Great
sound
from
Alexa
.
Play
games
.
Play
radio
stations
.
Play
old
time
radio
programs
like
the
Long
Ranger
.
Alexa
has
many
“
apps
”
you
can
add
to
her
abilities
.
The
other
day
I
played
Madlibs
.
Great
fun
.
Also
Alexa
can
control
household
appliances
.
I
just
purchased
Alexa
compatible
power
strips
to
control
lighting
,
fans
and
TV
components
.
I
’m
looking
forward
to
implementing
this
controll
.
You
can
also
add
other
Alexa
components
to
have
chosen
music
to
play
in
all
rooms
where
the
components
are
located
.
The
speaker
has
excellent
sound
and
works
perfectly
!
I
LIKE
IT
ALL
I
love
it
.
It
does
everything
I
was
a
little
nervous
when
I
received
my
new
Echo
as
I
'm
not
rea

it
's
annoying
that
everything
they
show
Alexa
as
being
able
to
do
has
to
be
downloaded
.
You
spend
so
 
much
time
trying
to
figure
out
how
to
phrase
it
for
the
machine
that
you
may
as
well
just
look
it
up
yourself
I
have
had
Alexa
since
the
beginning
.
 
I
accidentally
spilled
water
on
her
and
have
replaced
it
with
the
2nd
generation
.
 
I
decided
I
could
n't
be
without
an
Echo
.
 
Setup
was
easy
and
had
it
working
within
minutes
.
I
like
being
able
to
tell
Alexia
the
music
I
want
to
hear
,
but
sometimes
she
has
trouble
finding
the
song
if
I
am
not
very
specific
about
the
title
.
I
like
the
looks
better
than
the
original
echo
.
 
I
moved
my
old
echo
to
the
basement
family
room
.
Nice
sound
.
The
Echo
is
wonderful
but
I
'm
still
learning
how
to
use
it
.
Music
requests
are
somewhat
limited
.
Does
n't
know
half
the
things
asked
.
Ca
n't
recognize
my
sprinkler
wifi
controller
,
even
though
product
details
says
it
is
compatible
.
Ca
n't
program
or
change
my
wifi
thermostat
settings
and
its

speaker
sounds
good
Absolutely
love
!
!
The
best
part
of
this
product
is
you
can
control
the
thermostat
and
lights
for
your
house
.
 
There
is
n’t
anything
I
dislike
.
We
're
thoroughly
enjoying
our
Amazon
Echo
.
Installation
was
easy
using
the
Alexa
app
.
I
purchased
2
Kasa
smart
plugs
to
control
our
living
room
lights
.
Installation
again
was
easy
using
the
Kasa
app
.
They
sinqued
right
up
with
the
Echo
.
We
're
enjoying
playing
music
with
the
Echo
and
the
sound
quality
is
awesome
.
We
've
added
a
Kasa
smart
bulb
for
our
kitchen
.
Setup
was
seamless
.
We
're
looking
forward
to
adding
more
automation
to
our
home
.
Home
entertainment
Very
cool
product
.
Easy
to
setup
,
fun
to
use
and
sound
quality
is
much
better
than
I
anticipated
.
Love
these
,
 
great
sound
...
easy
to
connect
and
use
...
Does
n't
work
with
my
fairly
new
smart
TV
,
but
that
's
mainly
the
TV
manufacturers(LG
)
fault
due
to
not
supporting
anything
older
than
TVs
from
this
year
.
My
smart
locks
also
need
an
additional
p

Terrific
Love
my
Echo
It
’s
smaller
than
the
original
.
Looks
nicer
too
.
Works
great
!
My
husband
and
I
are
what
I
would
call
&
#
34;late
adopters&#34
;
when
it
come
to
technology
,
but
we
decide
we
would
try
and
Echo
to
serve
primarily
as
a
music
source
.
 
Wow
,
were
we
amazed
and
the
great
sound
!
 
We
've
also
been
having
a
great
time
listening
to
all
of
our
favorite
songs
buy
just
asking
Alexa
.
 
I
may
even
buy
one
for
my
elderly
Dad
-
I
think
he
will
enjoy
having
one
to
listen
to
music
or
even
place
his
calls
to
us
!
I
really
have
enjoyed
it
.
 
Changing
the
Directv
channels
,
plays
iHeartRadio
and
Pandora
,
adjusts
the
ac
thermostat
and
turns
the
living
room
lights
and
outside
lights
.
I
have
been
wanting
one
of
these
for
a
while
now
.
 
Bought
on
Prime
Day
and
absolutely
love
it
.
We
are
so
glad
that
we
got
the
Echo
over
a
Bluetooth
speaker
.
Alexa
does
so
much
more
stuff
than
other
non
smart
speakers
out
there
.
I
personally
love
the
timer
feature
.
It
has
helped
me
with
coo

...

I
loved
it
does
exactly
what
it
says
I
used
it
to
control
my
smart
home
devices
.
Works
great
.
Very
convenient
Este
producto
llegó
y
a
la
semana
se
quedó
sin
olor
,
perdió
toda
su
escencia
,
huele
solo
a
alcohol
Easy
to
set
up
Ready
to
use
in
minutes
.
Barry
 
My
three
year
old
loves
it
.
 
Good
for
doing
basic
household
tasks
.
 
We
're
going
to
add
a
few
smart
outlets
and
bulbs
.
Awesome
device
wish
I
bought
one
ages
ago
.
love
it
Perfect
for
kids
,
adults
and
everyone
in
between
!
!
Listening
to
music
,
searching
locations
,
checking
time
,
looking
up
weather
.
There
are
many
more
task
to
learn
.
It
was
simple
to
set
up
.
I
do
love
these
things
,
i
have
them
running
my
entire
home
,
TV
,
all
my
lights
,
my
thermostat
,
my
front
door
dead
bolt
,
the
cameras
...
and
i
love
listening
to
npr
.
these
things
are
well
worth
the
value
and
they
really
shine
once
you
stat
adding
smart
devices
and
you
can
control
your
entire
home
like
its
right
out
of
star
trek
....
I
named
mine
computer
to
g

I
love
having
Alexa
around
the
house
.
My
favorite
features
are
her
timers
,
alarms
,
and
music
.
Especially
her
library
of
sleep
sounds
.
I
got
this
product
as
a
gift
for
my
kids
,
and
they
love
her
too
.
She
is
great
for
reading
time
,
and
timing
30
minutes
for
it
.
The
only
thing
I
really
do
n't
like
,
is
that
she
HAS
to
be
plugged
in
to
an
outlet
to
function
.
No
ifs
ands
,
or
buts
.
I
get
why
,
but
it
is
minorly
inconvenient
,
as
I
have
to
move
her
from
room
to
room
,
or
purchase
more
.
Lol
Alarm
,
bed
time
story
,
universal
I
am
in
love
with
alexa
Its
was
easy
to
set
up
 
Works
better
than
anticipated
.
This
thing
is
awesome
.
Great
product
Really
like
it
I
like
that
it
respond
most
of
my
comands
.
Great
product
.
The
volume
is
very
low
so
far
i
am
liking
it
although
i
am
probably
not
using
it
to
the
full
capasity
My
new
best
friend
!
!
I
'm
a
little
behind
the
times
I
've
never
had
anything
like
this
,
another
alexa
or
google
home
,
so
to
me
this
is
a
super
amazing
useful
tool
.

Alexa
is
phenomenal
!
When
you
think
about
it
this
really
does
n’t
do
much
.
Play
music
,
answer
stupid
questions
,
finally
it
was
able
to
hook
up
to
thermostat
.
I
can
see
why
price
is
dropping
like
a
rock
.
The
Echo
Dot
(
2nd
Generation
)
Smart
Speaker
is
very
compound
in
size
,
however
it
 
projects
very
good
sound
.
It
also
has
a
port
in
which
you
can
connect
an
external
speaker
 
for
greater
projection
of
sound
.
I
also
like
it
size
because
it
's
not
awkward
,
and
it
does
n't
take
up
a
lot
of
space
.
It
works
great
with
the
Alexa
App
.
I
also
think
the
size
is
great
for
college
students
who
live
in
dorm
rooms
with
small
limited
spacing
.
I
use
the
echo
dot
several
different
ways
in
my
home
.
I
use
it
downstairs
to
control
lighting
.
I
also
take
it
out
on
my
deck
to
stream
music
to
a
Bluetooth
speaker
.
It
works
great
for
what
I
need
.
I
have
an
echo
but
wanted
a
dot
for
my
bedroom
.
It
was
even
easier
to
set
up
than
my
original
echo
.
Makes
turning
on
and
off
things
in
the
house
s

Better
sound
.
Got
this
for
my
2nd
Echo
in
the
house
,
I
already
have
the
regular
Echo
.
 
The
Echo
Dot
was
easy
to
set
up
but
the
sound
is
not
enjoyable
,
wishing
I
would
have
just
gone
with
another
regular
Echo
solely
based
on
sound
.
I
was
really
happy
with
my
original
echo
so
i
thought
I
'd
get
an
echo
dot
to
use
in
my
bedroom
.
I
was
really
disappointed
in
the
audio
quality
so
I
connected
an
external
speaker
via
bluetooth
.
The
audio
was
much
better
but
I
started
having
problems
with
it
loosing
connection
with
the
wifi
,
presumably
due
to
interference
from
the
bluetooth
.
Then
 
I
connected
a
speaker
via
the
auxiliary
jack
.
when
i
did
that
,
the
auxiliary
jack
picked
up
interference
from
the
wifi
and
I
was
woken
up
in
the
middle
of
the
night
by
a
horrible
buzzing
sound
.
i
m
hoping
Amazon
will
take
this
thing
back
and
give
me
a
good
deal
on
an
echo
spot
which
I
hope
will
be
a
better
nightstand
device
.
Listens
well
,
does
way
more
than
expected
,
love
her
!
We
are
still
testing
t

Great
.
Definitely
gets
a
little
old
after
a
while
but
for
$
30
I
ca
n't
really
regret
it
at
all
.
Love
having
it
hooked
up
to
my
speakers
via
bluetooth
and
waking
up
to
The
Smiths
.
Love
it
.
Added
it
to
an
upstairs
bedroom
Got
to
love
Alexa
she
does
everything
around
our
house
Works
great
and
easy
to
set
up
!
My
3rd
echo
device
.
Very
easy
to
set
up
.
Works
perfectly
.
Easy
to
set
up
and
start
using
..
works
as
advertised
.
 
Love
our
little
dot
.
I
do
n’t
know
what
the
original
larger
Alexa
can
do
that
the
dot
can
not
do
but
we
are
pleased
with
our
dots
capabilities
.
Really
like
it
Works
as
expected
.
This
is
our
fourth
one
,
our
son
loves
it
in
his
room
.
Easy
to
set
an
alarm
and
he
can
play
music
through
it
.
Love
it
Dislike
the
volume
.
It
does
not
sound
loud
enough
.
Love
Alexa
!
!
 
I
own
2
and
gave
one
for
a
gift
which
was
a
huge
hit
!
as
expected
I
did
n’t
order
it
I
love
this
technology
.
I
'm
older
,
but
this
is
so
easy
to
set
up
and
use
.
I
have
the
dot
in
my
bedroom
and
